In [ ]:
import matplotlib.pyplot as plt
import xlsxwriter
import numpy as np
import pandas as pd
import pandas.io.sql
import os
import pdfkit
import pytz 
from datetime import datetime 
import base64
import pymssql
import psycopg2 as pg
from jinja2 import Environment, FileSystemLoader

host        = '192.168.1.13'
database    = 'igu_live'
user        = 'sa'
password    = 'B1admin'

host2        = '192.168.1.13'
database2    = 'igu_webapp'
user2        = 'sa'
password2    = 'B1admin'

 
conn = pymssql.connect(host=host, user=user, password=password, database=database)
conn2 = pymssql.connect(host=host2, user=user2, password=password2, database=database2)

msg_sql= """                
                select    
                        c.cardcode  ,
                        c.cardname 'Partner Name',
                        d.groupname 'Group',
                        c.U_AR_Person , 
                        sum(case 
                                when datediff(day,a.refdate,getdate())<=30 and a.transtype in (13,14) then (a.BalScDeb -a.balsccred ) 
                                else 0
                        end) '0-30',  
                        sum(case 
                                when datediff(day,a.refdate,getdate()) between 31 and 60  and a.transtype in (13,14)  then (a.BalScDeb -a.balsccred ) 
                                else 0
                        end) '31-60',  
                        sum(case 
                                when datediff(day,a.refdate,getdate())  between 61 and 90  and a.transtype in (13,14)  then (a.BalScDeb -a.balsccred ) 
                                else 0
                        end) '61-90',  
                        sum(case 
                                when datediff(day,a.refdate,getdate())  between 91 and 120    and a.transtype in (13,14)  then (a.BalScDeb -a.balsccred ) 
                                else 0
                        end) '91-120',  
                        sum(case 
                                when datediff(day,a.refdate,getdate()) >=121   and a.transtype in (13,14)  then (a.BalScDeb -a.balsccred ) 
                                else 0
                        end) '121+',  
                        
                        sum(a.BalScDeb -a.balsccred ) 'Total'
                from jdt1 a 
                inner join ojdt b on a.transid = b.transid 
                inner join ocrd c on a.ShortName = c.cardcode 
                inner join ocrg d on d.groupcode = c.groupcode 
                where 
                        a.account ='1130001' 
                        and a.BalScDeb -a.balsccred  <>0 
                        and convert(varchar,a.refdate,112)<= convert(varchar,getdate(),112)
                group by  
                        c.cardcode ,
                        d.groupname ,
                        c.cardname ,
                        c.U_AR_Person   
                
        """

pandas.options.display.float_format = '{:,.2f}'.format

#msg_sql = """ exec [dbo].[IGU_AGING_AR] '20200531','IGU' """
data = pandas.io.sql.read_sql(msg_sql,conn)

msg_sql2= """
              select  a.id [Key],
                        c.Name itype,
                        a.TransNo,
                        coalesce(a.remarks,'') remarks,
                        convert(varchar,a.transdate,23) transdate,
                        coalesce(a.Collector ,'') Collector,
                        d.UserName,
                        a.amountDetailLc ,
                        a.CustomerCode ,
                        a.CustomerName ,
                        a.PayToCode ,
                        convert(varchar,b.InvoiceDate ,23) invoiceDate,
                        b.InvoiceTypeName,
                        b.InvoiceRefNo,
                        b.InvoiceKwitansiNo ,
                        b.InvoiceFakturNo ,
                        b.InvoiceAmountTc 
                from Tx_PaymentIn a 
                    inner join Tx_PaymentIn_ArCorporate b on a.Id = b.Id
                    inner join Tm_User  d on a.ModifiedUser = d.Id
                    LEFT OUTER JOIN Ts_List c ON c.[Type] = 'PaymentInRefType' AND a.ReferenceType = c.Code
                where a.ReferenceType<>'Payment'
                and convert(varchar,A.transdate ,112) = '20200801'
        """


data2 = pandas.io.sql.read_sql(msg_sql2,conn2)

 
#data
data
data2
#new_df = pandas.merge(data, data2,  on=["cardcode"])

#new_df.to_excel("followup.xlsx")
#
# new_df[["cardcode"]]
#print(data)
#data3 =pandas.merge()


